In [1]:
%matplotlib inline
import gym
import gym_vgdl
import numpy as np

from gym import wrappers

import time

# Q-learning
import itertools
import pandas as pd
import sys

from collections import defaultdict
from lib import plotting


In [3]:
env = gym.make('vgdl_aaa-v0')
# env = gym.make('vgdl_aliens-v0')

UnregisteredEnv: No registered env with id: vgdl_aaa-v0

In [9]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy_fn(observation):
        A = np.ones(nA, dtype=float)* epsilon/nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn


In [10]:
def convert_state(x, y):
    return (x-1)*19+y

In [11]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    Args:
        alpha: TD learning rate
    """

    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))

    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)

    for i_episode in range(num_episodes):
        # env.render()
        # time.sleep(0.1)
        if(i_episode+1) % 100 == 0:
            print("\rEpisode {}/{}.".format(i_episode+1, num_episodes), end="")
            sys.stdout.flush()

        state = env.reset()
        # print("initial state ", state)
        state_int = convert_state(state[1], state[0])
        # print("initial state ", state_int)
        for t in itertools.count():
            # env.render()
            # time.sleep(0.1)
            # Take a step
            action_probs = policy(state_int)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            # action = env.action_space.sample()
            # print("action ", action)
            next_state, reward, done, _ = env.step(action)
            # print("next_state ", next_state)
            next_state_int = convert_state(next_state[1], next_state[0])
            # print("next_state_int", next_state_int)
            # Update stats
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t

            # TD Update
            best_next_action = np.argmax(Q[next_state_int])
            td_target = reward + discount_factor*Q[next_state_int][best_next_action]
            td_delta = td_target - Q[state_int][action]
            Q[state_int][action] += alpha * td_delta

            if done:
                break
            state = next_state

    return Q, stats

In [12]:
Q, stats = q_learning(env, 500)
# Q = q_learning(env, 500)
plotting.plot_episode_stats(stats)

NameError: name 'env' is not defined